In [ ]:
import yaml
import json

In [ ]:
with open("demosaic_survey.yml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

print(json.dumps(data,indent=4))